<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Задание" data-toc-modified-id="Задание-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Задание</a></span></li><li><span><a href="#Задание" data-toc-modified-id="Задание-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Задание</a></span></li><li><span><a href="#Выполнение-задания" data-toc-modified-id="Выполнение-задания-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выполнение задания</a></span><ul class="toc-item"><li><span><a href="#Задание-1" data-toc-modified-id="Задание-1-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Задание 1</a></span></li></ul></li><li><span><a href="#Данные-по-определенному-объекту" data-toc-modified-id="Данные-по-определенному-объекту-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Данные по определенному объекту</a></span></li></ul></div>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ds-reboot/python-first-part/blob/main/notebooks/hometasks/Task1_dmrf.ipynb)

# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание 
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


# Выполнение задания

## Задание 1

In [1]:
import requests
import tqdm
import pandas as pd
import sqlite3

In [16]:
class DomIdLoader:
    def __init__(self):
        self.offset_ = 1
        self.limit_ = 1000
        self.objids = []
        self.all_data_domrf = []
        self.page = ''
        self.data_df = None


    def get_objId(self):
        for _ in range(15):
            try:
                url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={self.offset_}&limit={self.limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
                res = (requests.get(url))
                objects_data = (res.json())
                objects_data.get('data').get('list')
                objects_list = objects_data.get('data').get('list')
                self.objids += [x.get('objId') for x in objects_list]
                self.offset_ += 1000
                self.limit_ += 1000
            except:
                print('No more data')
                self.objids = set(self.objids)
        self.objids = list(self.objids)
        print("ID объектов с сайта ДОМ РФ собраны!")

    def get_data(self):
        for i in (range(len(self.objids))):
            url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{self.objids[i]}'
            res = requests.get(url)
            res = res.json()
            all_data = res.get('data')
            if 'developer' in all_data:
                all_data.update(all_data['developer'])
                all_data.pop('developer')
            if 'quartography' in all_data:
                all_data.update(all_data['quartography'])
                all_data.pop('quartography')
            self.all_data_domrf.append(all_data)
        print("Выгружены данные по объектам")

    def make_data_dataframe(self):
        print("Создан DataFrame")
        return pd.DataFrame(self.all_data_domrf)
        
    
    def get_df_with_data(self):
        self.get_objId()
        self.get_data()
        self.make_data_dataframe()
        self.data_df = self.make_data_dataframe()
        return self.data_df

In [17]:
DomIdLoader = DomIdLoader()
Data = DomIdLoader.get_df_with_data()

No more data
No more data
No more data
No more data
No more data
No more data
No more data
No more data
No more data
No more data
No more data
No more data
No more data
ID объектов с сайта ДОМ РФ собраны!
Выгружены данные по объектам
Создан DataFrame
Создан DataFrame


In [15]:
len(Data)

1000

In [5]:
class Saver:
    def __init__(self, data):
        self.data = data
        self.file_name = 'Data'
        self.conn = sqlite3.connect('skills_database')        

    def save_csv(self):
        self.data.to_csv(f'{self.file_name}.csv', index=False)
        print(f'Dataset сохранён в файл {self.file_name}.csv')

    def save_xl(self):
        self.data.to_excel(f'{self.file_name}.xlsx', index=False)
        print(f'Dataset сохранён в файл {self.file_name}.xlsx')

    def save_sql(self):
        self.data.to_sql(f'{self.file_name}', self.conn, if_exists='replace', index = False)
        self.conn.commit()
        self.conn.close()
        print(f'Dataset сохранён в файл {self.file_name}.csv')
        
    def save_pcl(self):
        self.data.to_pickle(f'{self.file_name}.pkl')
        print(f'Dataset сохранён в файл {self.file_name}.pkl')

In [18]:
Saver_ = Saver(Data)
Saver_.save_xl()

Dataset сохранён в файл Data.xlsx


In [19]:
df = pd.read_csv('Data.csv')

In [20]:
len(df)

7000

In [ ]:
class PhotoLoader:
    def __init__(self):
        self.offset_ = 1
        self.limit_ = 10
        self.all_photo_domrf = []
        self.objids = []
        self.page = ''
        self.name_photo = 0

    def get_objId(self):
        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={self.offset_}&limit={self.limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
        res = (requests.get(url))
        objects_data = (res.json())
        objects_data.get('data').get('list')
        objects_list = objects_data.get('data').get('list')
        objids = [x.get('objId') for x in objects_list]
        self.objids.extend(objids)
        print("ID объектов с сайта ДОМ РФ собраны!")

    def get_photo_url(self):
        for i in (range(len(self.objids))):
            url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{self.objids[i]}'
            res = requests.get(url)
            res = res.json()
            all_photo = res.get('data').get('photoRenderDTO')
            for i in all_photo:
                self.all_photo_domrf.append(i.get('objRenderPhotoUrl'))
        print("Выгружены url фотографий в list")
        
    def get_photo(self):
        for img in self.all_photo_domrf:
            self.name_photo += 1
            p = requests.get(img)
            out = open(f"{self.name_photo}_photo.jpg", "wb")
            out.write(p.content)
            out.close()
        print("Фотографии выгружены")

    def get_all_photo(self):
        self.get_objId()
        self.get_photo_url()
        self.get_photo()

In [ ]:
PhotoLoader = PhotoLoader()
Photo = PhotoLoader.get_all_photo()